In [2]:
# import statsapi
import sqlalchemy
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time, os
from selenium import webdriver

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
!pip install statsapi



ERROR: Could not find a version that satisfies the requirement statsapi (from versions: none)
ERROR: No matching distribution found for statsapi


In [ ]:
# def division_standings(league, division):
#     standings = pd.DataFrame(statsapi.standings_data(league)[division]['teams'])
#     standings = standings[['name', 'div_rank', 'gb', 'w','l']]
#     return standings
# division_standings(104, 204)

In [ ]:
# print(statsapi.league_leaders('homeruns',statGroup='hitting',limit=5,season=1980) )

In [ ]:
# pip install requests beautifulsoup4

In [ ]:
# page = requests.get('hhttps://www.baseball-reference.com/leagues/MLB/bat.shtml’)
# # this doesnt work for baseball reference, have to use selenium with dynamic content on pages

In [ ]:
# driver = webdriver.Chrome('/usr/local/bin/chromedriver-2')  # Optional argument, if not specified will search path.
# driver.get('http://www.google.com/');
# time.sleep(5) # Let the user actually see something!
# search_box = driver.find_element_by_name('q')
# search_box.send_keys('ChromeDriver')
# search_box.submit()
# time.sleep(5) # Let the user actually see something!


In [ ]:
# driver.get('https://www.baseball-reference.com/leagues/MLB/2016-starter-pitching.shtml')
# time.sleep(5)
# driver.execute_script("window.scrollTo(0, 1500);")

In [ ]:
def get_csv_bbref(link_list):
    soup_list = []
    for i in link_list:
        driver = webdriver.Chrome(chromedriver)
        driver.get(i)
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, 1500);")
        driver.find_element_by_xpath('//*[@id="all_players_starter_pitching"]/div[1]/div/ul/li[1]').click()
        time.sleep(5)
        driver.find_element_by_xpath('//*[@id="all_players_starter_pitching"]/div[1]/div/ul/li[1]/div/ul/li[4]/button').click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        crude = soup.find('pre', id = 'csv_players_starter_pitching').text.split("\n")
        soup_list.append(crude)
    return soup_list

In [ ]:
#get all pages at once
bbref_list = ['https://www.baseball-reference.com/leagues/MLB/2016-starter-pitching.shtml', 
              'https://www.baseball-reference.com/leagues/MLB/2017-starter-pitching.shtml',
              'https://www.baseball-reference.com/leagues/MLB/2018-starter-pitching.shtml',
              'https://www.baseball-reference.com/leagues/MLB/2019-starter-pitching.shtml']

In [ ]:
raw_csv_data_list = get_csv_bbref(bbref_list)


In [ ]:
def clean_crude(csv):
    no_blank = [] 
    for i in csv:
        for j in i:
            if j != '':
                no_blank.append(j)
        new_data_split = []
    for i in no_blank:
        new_line = pd.DataFrame(i.split(','))
        new_data_split.append(new_line)
    season_data = pd.concat(new_data_split, ignore_index = True, axis = 1).T
    header = season_data.iloc[0]
    season_data = season_data[1:]
    season_data.columns = header
    season_data.drop(columns = ['Rk'], inplace = True)
    return season_data

In [ ]:
# clean_2016 = clean_crude(raw_csv_data_2016)
clean_2016.to_csv('season_2016.csv', index = False)